En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Solucion Challenge Latam
## Nelson David Navarro Diaz

Nota. La explicacion detalla de cada una de las operaciones hara parte de los comentarios de cada funcion, los comentarios se haran en ingles entendiendo las buenas practicas de comentar en este idioma, debido a que este idioma no nos limitaria a compartirlo con personas no hispanohablantes. Este documento si se hace en espanol por ser el idioma de los involucrados en este proceso de seleccion.

Antes de inicial el desarrollo de cada uno de los casos aplicando es necesario hacer una revision de los datos que tenemos. Esto debido a que el entender el negocio y las condiciones de los datos que se estaran procesando es el primera paso para desarrollar una solucion.  Ya que el entender el origen, la naturaleza de los datos y la aplicacion que le vamos a dar a estos es lo que finalmente nos permitira tomar las decisiones adecuadas en el desarrollo de una solucion.  En este caso particular tenemos los siguientes insigths. 
-  La informacion tiene como origen la plataforma X antes twitter y al ser la informacion publicada por estos no solo tiene la informacion necesaria en cada caso, sino que tambien esta cargada de informacion adicional.
- Aunque en este momento tenemos un set de datos limitados este podria seguir creciendo indefinidamente ya que esta informacion se genera en tiempo real en esta aplicacion. 
- La informacion ingresa en un formato Json con Child Json, por lo que alguna informacion no  podra ser leida en el primer nivel de exploracion del Json. 



## Q1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:


1. Optimizacion del tiempo de ejecucion.
Incialmente considerando que la informacion viene  en formato Json el primer paso es transformarlo a un tipo de datos mucho mas facil de manejar. En este caso se procede a pasaralo a dataframe de pyspark. Debido a que este formato es columnar lo que al trabajar con grandes datos nos permite utilizar unicamente las colummnas que requerimos lo que hace que se trabaje con menos informacion. En este caso como se busca la optimizacion del tiempo se haran los procesos utilizando la totalidad del archivo  lo que hara que cada una de las operaciones se haga de una vez sobre la totalidad del archivo.

AHora procedemos a llamar la funcion, inicialmente medimos tiempo de ejecucion, memoria y vemos el resultado la ejecucion

In [3]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [5]:
from q1_time import q1_time
%timeit -r 1 -n 1 result_list = q1_time(file_path)

Filename: c:\Users\Nelson\Documents\latam_challenge\src\q1_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    12     78.6 MiB     78.6 MiB           1   @profile
    13                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    14                                                 #El proceso se hace con spark para trabajar la informacion de manera columnar. 
    15     80.0 MiB      1.4 MiB           1           spark = SparkSession.builder.appName('sparkdf').getOrCreate() 
    16     80.0 MiB      0.0 MiB           1           df = spark.read.json(file_path)
    17     80.0 MiB      0.0 MiB           1           df = df.withColumn("date", col("date").substr(1, 10))
    18     80.0 MiB      0.0 MiB           1           df = df.select("date", col("user.username").alias("user")).dropna()
    19                                                 
    20                                         
    21     80.1 MiB      0.

In [ ]:
from memory_profiler import memory_usage
from q1_time import q1_time
mem_usage = memory_usage((q1_time, (file_path,)), interval=1, timeout=None)
print(f"Memory usage: {max(mem_usage)} MiB")

Memory usage: 550.62890625 MiB


In [ ]:
result_list = q1_time(file_path)
print(result_list)

[('2021-02-18', 'neetuanjle_nitu'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-14', 'rebelpacifist'), ('2021-02-15', 'jot__b'), ('2021-02-23', 'Surrypuria'), ('2021-02-19', 'Preetm91'), ('2021-02-12', 'RanbirS00614606')]


2. Optimizacion del uso Memoria:  Para este caso de aplicacion particular buscando optimizar la memoria se procesara por chunks o fragmentos del archivo inicial esto con la finalidad de tener dataframes que ocupen menos espacio ya que en cada lectura solamente se tendria acumulado un fragmento del archivo en memoria, se evaluaria y se almacena unicamente el resultado de esa transaccion  al final se hace una exploracion conjunta de los resultados. De tal manera que se tendrian los resultados consolidado. Esto requiere mayor tiempo de procesamiento pero el uso de memoria se mantendra bajo. Bajo este mismo concepto se maneja tambien en aplicaciones real time, lo cual podria ser conveniente teniendo en cuenta la plataforma de origen de estos datos, ya que una implementacion como esta nos permitira  leer en tiempo real la informacion que se va generando y al final ir viendo como va cambiando los insights.

In [ ]:
from memory_profiler import memory_usage
from q1_memory import q1_memory
mem_usage = memory_usage((q1_memory, (file_path,)), interval=1, timeout=None)
print(f"Memory usage: {max(mem_usage)} MiB")

Memory usage: 144.08203125 MiB


In [ ]:
from q1_memory import q1_memory
%timeit -r 1 -n 1 result_list = q1_memory(file_path)

40.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
result_list = q1_memory(file_path)
print(result_list)

[('2021-02-18', 'neetuanjle_nitu'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-14', 'rebelpacifist'), ('2021-02-15', 'jot__b'), ('2021-02-23', 'Surrypuria'), ('2021-02-19', 'Preetm91'), ('2021-02-12', 'RanbirS00614606')]


Conclusiones: Se ve que en la primera manera se hace en menor tiempo y en la segunda se reduce el consumo de memoria si quisieramos mejorar el uso de memoria hariamos el tamano del fragmento a explorar mas pequeno pero tardariamos mas tiempo. Ahi debemos jugar con los recursos disponibles para tomar una decision en este caso vemos que un tamano de 100 000 lineas no se alarga demasiado el procesamiento 

## Los top 10 emojis más usados con su respectivo conteo. 


1. En este caso particular se encuentra la posibilidad de que solo se debe leer un valor del json para esto se lee en una lista y luego se crea un solo string para alamacenar todo, esto debido a que no nos interesa saber quien dio la informacion solo que se repita y el procesarlo como un string nos da la ventaja de trabajar con un solo gran bloque de texto sin estructura que nos reducce el procesamiento. 

In [ ]:
from q2_time import q2_time
%timeit -r 1 -n 1 result_list = q2_time(file_path)

6.22 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
from memory_profiler import memory_usage
from q2_time import q2_time
mem_usage = memory_usage((q2_time, (file_path,)), interval=1, timeout=None)
print(f"Memory usage: {max(mem_usage)} MiB")

Memory usage: 573.26171875 MiB


In [ ]:
result= q2_time(file_path)
print(result)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


2. memory optimization. En este caso se toma el mismo abordaje que en el primero, se hace la division por strings mas pequenos teniendo asi un procesamiento mas pequeno de los datos que se estan manejando.  Siguiendo el mismo concepto de los chunks del punto anterior

In [6]:
from q2_memory import q2_memory
%timeit -r 1 -n 1 result_list = q2_memory(file_path)

6.23 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
from memory_profiler import memory_usage
from q2_memory import q2_memory
mem_usage = memory_usage((q2_memory, (file_path,)), interval=1, timeout=None)
print(f"Memory usage: {max(mem_usage)} MiB")

Memory usage: 544.1328125 MiB


In [7]:
from q2_memory import q2_memory
result= q2_memory(file_path)
print(result)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


Conclusiones. En este caso al no tener datos que sean de estructura compleja entre una alternativa y otra no se tiene una diferencia significativa como si se noto en el caso anterior. Esto debido a que se estaba trabajando solo con el texto que resulta ser mas ligero de procesar. 